## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Youghal,IE,51.9500,-7.8506,41.00,87,20,6.91,few clouds
1,1,Lavrentiya,RU,65.5833,-171.0000,6.12,89,100,10.40,overcast clouds
2,2,Busselton,AU,-33.6500,115.3333,64.99,89,100,5.01,light rain
3,3,Quthing,LS,-30.4000,27.7003,53.37,51,3,2.06,clear sky
4,4,Qaanaaq,GL,77.4840,-69.3632,-20.63,77,35,7.47,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Vila Velha,BR,-20.3297,-40.2925,75.20,88,75,4.61,broken clouds
7,7,Hilo,US,19.7297,-155.0900,73.00,83,90,11.50,moderate rain
10,10,Tual,ID,-5.6667,132.7500,83.62,69,19,3.69,few clouds
11,11,Puerto Ayora,EC,-0.7393,-90.3518,77.00,94,48,5.99,scattered clouds
15,15,Katherine,AU,-14.4667,132.2667,82.40,61,100,12.66,overcast clouds
17,17,Rikitea,PF,-23.1203,-134.9692,77.95,62,3,17.49,clear sky
21,21,Vaini,TO,-21.2000,-175.2000,77.00,78,90,9.22,overcast clouds
22,22,Salalah,OM,17.0151,54.0924,75.20,73,0,4.61,clear sky
25,25,Adrar,MR,20.5022,-10.0711,79.23,22,32,8.12,scattered clouds
26,26,Atuona,PF,-9.8000,-139.0333,80.69,73,100,11.01,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Vila Velha,BR,75.20,broken clouds,-20.3297,-40.2925,
7,Hilo,US,73.00,moderate rain,19.7297,-155.0900,
10,Tual,ID,83.62,few clouds,-5.6667,132.7500,
11,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,
15,Katherine,AU,82.40,overcast clouds,-14.4667,132.2667,
17,Rikitea,PF,77.95,clear sky,-23.1203,-134.9692,
21,Vaini,TO,77.00,overcast clouds,-21.2000,-175.2000,
22,Salalah,OM,75.20,clear sky,17.0151,54.0924,
25,Adrar,MR,79.23,scattered clouds,20.5022,-10.0711,
26,Atuona,PF,80.69,overcast clouds,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Vila Velha,BR,75.20,broken clouds,-20.3297,-40.2925,Quality Suites Vila Velha
7,Hilo,US,73.00,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Tual,ID,83.62,few clouds,-5.6667,132.7500,Grand Vilia Hotel
11,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
15,Katherine,AU,82.40,overcast clouds,-14.4667,132.2667,Pine Tree Motel
17,Rikitea,PF,77.95,clear sky,-23.1203,-134.9692,Pension Maro'i
21,Vaini,TO,77.00,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
22,Salalah,OM,75.20,clear sky,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
25,Adrar,MR,79.23,scattered clouds,20.5022,-10.0711,
26,Atuona,PF,80.69,overcast clouds,-9.8000,-139.0333,Villa Enata


In [37]:
clean_hotel_df = clean_hotel_df[clean_hotel_df["Hotel Name"] != ""]

In [38]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Vila Velha,BR,75.20,broken clouds,-20.3297,-40.2925,Quality Suites Vila Velha
7,Hilo,US,73.00,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
10,Tual,ID,83.62,few clouds,-5.6667,132.7500,Grand Vilia Hotel
11,Puerto Ayora,EC,77.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
15,Katherine,AU,82.40,overcast clouds,-14.4667,132.2667,Pine Tree Motel
17,Rikitea,PF,77.95,clear sky,-23.1203,-134.9692,Pension Maro'i
21,Vaini,TO,77.00,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
22,Salalah,OM,75.20,clear sky,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
26,Atuona,PF,80.69,overcast clouds,-9.8000,-139.0333,Villa Enata
30,Lincoln,US,73.40,clear sky,40.8000,-96.6670,Courtyard by Marriott Lincoln Downtown/Haymarket


In [39]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))